In [1]:
import os
import copy
import numpy as np
import pandas as pd
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터 확인

In [2]:
numeric = pd.read_csv('./data/train_numeric.csv')
numeric.shape

(10000, 970)

In [3]:
categorical = pd.read_csv('./data/train_categorical.csv')
categorical.shape

(10000, 2141)

In [4]:
numeric.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
categorical.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- numeric/categorical data로 구분
- 데이터샘플은 적고 특징(컬럼)이 굉장히 많은 데이터로써 특징 추출이 매우 중요한 문제
- 결측치가 매우 많다.
- 비식별화된 특징이 매우 많다
- 불량 예측문제 답게 클래스 불균형 문제가 심각

- ID별 - L(제조 라인)_S(제조 스테이션)_F(기능 번호)

# 데이터 EDA & 전처리

## 수치형 데이터

In [6]:
df = numeric.copy()

In [7]:
df.set_index('Id', inplace=True)

X = df.drop('Response', axis=1)
Y = df['Response']

### 라인별 스테이션고 특징 확인

- 4개의 라인이 존재 : L0 ~ L3

In [8]:
X.head()

,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,...,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
Id,,,,,,,,,,,,,,,,,,,,,
4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,0.083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,-0.147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,-0.046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,-0.099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
line_station_feature_dict = dict()

line_station_feature_dict['L0'] = {'station':[], 'feature':[]}
line_station_feature_dict['L1'] = {'station':[], 'feature':[]}
line_station_feature_dict['L2'] = {'station':[], 'feature':[]}
line_station_feature_dict['L3'] = {'station':[], 'feature':[]}

In [10]:
X.columns

Index(['L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8', 'L0_S0_F10',
       'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16', 'L0_S0_F18',
       ...
       'L3_S50_F4243', 'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249',
       'L3_S50_F4251', 'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258',
       'L3_S51_F4260', 'L3_S51_F4262'],
      dtype='object', length=968)

In [11]:
for col in X.columns:
    line, station, feature = col.split('_')

    if station not in line_station_feature_dict[line]['station']:
        line_station_feature_dict[line]['station'].append(station)
    
    if feature not in line_station_feature_dict[line]['feature']:
        line_station_feature_dict[line]['feature'].append(feature)

In [13]:
for line in line_station_feature_dict.keys():
    print(line_station_feature_dict[line]['station'])

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23']
['S24', 'S25']
['S26', 'S27', 'S28']
['S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S43', 'S44', 'S45', 'S47', 'S48', 'S49', 'S50', 'S51']


In [14]:
for line in line_station_feature_dict.keys():
    print(line_station_feature_dict[line]['feature'])

['F0', 'F2', 'F4', 'F6', 'F8', 'F10', 'F12', 'F14', 'F16', 'F18', 'F20', 'F22', 'F24', 'F28', 'F32', 'F36', 'F40', 'F44', 'F48', 'F52', 'F56', 'F60', 'F64', 'F68', 'F72', 'F76', 'F80', 'F84', 'F88', 'F92', 'F96', 'F100', 'F104', 'F109', 'F114', 'F116', 'F118', 'F122', 'F132', 'F136', 'F138', 'F142', 'F144', 'F146', 'F149', 'F155', 'F160', 'F165', 'F170', 'F175', 'F180', 'F185', 'F190', 'F195', 'F200', 'F205', 'F210', 'F219', 'F224', 'F229', 'F234', 'F239', 'F244', 'F249', 'F254', 'F259', 'F264', 'F269', 'F274', 'F282', 'F286', 'F290', 'F294', 'F298', 'F302', 'F306', 'F310', 'F314', 'F318', 'F322', 'F326', 'F330', 'F332', 'F334', 'F336', 'F338', 'F340', 'F342', 'F344', 'F346', 'F348', 'F350', 'F352', 'F354', 'F356', 'F358', 'F362', 'F366', 'F370', 'F374', 'F378', 'F382', 'F386', 'F390', 'F394', 'F397', 'F400', 'F403', 'F406', 'F409', 'F412', 'F415', 'F418', 'F421', 'F426', 'F431', 'F433', 'F435', 'F439', 'F449', 'F453', 'F455', 'F459', 'F461', 'F463', 'F466', 'F472', 'F477', 'F482', 'F4

- 제품별 결측이 아닌 라인 추출

In [17]:
X.iloc[0].notnull()

L0_S0_F0         True
L0_S0_F2         True
L0_S0_F4         True
L0_S0_F6         True
L0_S0_F8         True
                ...  
L3_S50_F4253    False
L3_S51_F4256    False
L3_S51_F4258    False
L3_S51_F4260    False
L3_S51_F4262    False
Name: 4, Length: 968, dtype: bool

In [18]:
not_null_columns = X.columns[X.iloc[0].notnull()]

In [19]:
pd.Series(not_null_columns)

0          L0_S0_F0
1          L0_S0_F2
2          L0_S0_F4
3          L0_S0_F6
4          L0_S0_F8
           ...     
157    L3_S35_F3913
158    L3_S37_F3944
159    L3_S37_F3946
160    L3_S37_F3948
161    L3_S37_F3950
Length: 162, dtype: object

In [20]:
pd.Series(not_null_columns).str.split('_', expand=True)

,0,1,2
0,L0,S0,F0
1,L0,S0,F2
2,L0,S0,F4
3,L0,S0,F6
4,L0,S0,F8
...,...,...,...
157,L3,S35,F3913
158,L3,S37,F3944
159,L3,S37,F3946
160,L3,S37,F3948


In [23]:
num_iter = 0

for idx, row in X.iterrows():
    if sum(row.notnull()) > 0:
        not_null_columns = X.columns[row.notnull()]
        lines = pd.Series(not_null_columns).str.split('_', expand=True).iloc[:, 0].drop_duplicates().to_list()
        stations = pd.Series(not_null_columns).str.split('_', expand=True).iloc[:, 1].drop_duplicates().to_list()
        features = pd.Series(not_null_columns).str.split('_', expand=True).iloc[:, 2].drop_duplicates().to_list()

        print(idx, lines, stations, features)

        if num_iter > 10:
            break
    num_iter += 1

4 ['L0', 'L3'] ['S0', 'S1', 'S2', 'S4', 'S7', 'S8', 'S11', 'S29', 'S30', 'S31', 'S33', 'S34', 'S35', 'S37'] ['F0', 'F2', 'F4', 'F6', 'F8', 'F10', 'F12', 'F14', 'F16', 'F18', 'F20', 'F22', 'F24', 'F28', 'F32', 'F36', 'F40', 'F44', 'F48', 'F52', 'F56', 'F60', 'F64', 'F104', 'F109', 'F136', 'F138', 'F142', 'F144', 'F146', 'F149', 'F282', 'F286', 'F290', 'F294', 'F298', 'F302', 'F306', 'F310', 'F314', 'F318', 'F322', 'F326', 'F3315', 'F3318', 'F3321', 'F3324', 'F3327', 'F3330', 'F3333', 'F3336', 'F3339', 'F3342', 'F3345', 'F3348', 'F3351', 'F3354', 'F3357', 'F3360', 'F3367', 'F3370', 'F3373', 'F3376', 'F3379', 'F3382', 'F3385', 'F3388', 'F3395', 'F3398', 'F3401', 'F3404', 'F3407', 'F3412', 'F3421', 'F3424', 'F3427', 'F3430', 'F3433', 'F3436', 'F3439', 'F3442', 'F3449', 'F3452', 'F3455', 'F3458', 'F3461', 'F3464', 'F3467', 'F3470', 'F3473', 'F3476', 'F3479', 'F3482', 'F3485', 'F3488', 'F3491', 'F3494', 'F3499', 'F3504', 'F3509', 'F3514', 'F3519', 'F3524', 'F3529', 'F3534', 'F3539', 'F3544',

### 'Id'별 거쳐간 스테이션 확인

In [24]:
total_stations = []

for line in line_station_feature_dict.keys():
    total_stations += line_station_feature_dict[line]['station']

print(total_stations)

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S43', 'S44', 'S45', 'S47', 'S48', 'S49', 'S50', 'S51']


In [29]:
np.isin([1, 2, 3], [1, 2])

array([ True,  True, False])

In [26]:
station_x = []

for idx, row in X.iterrows():
    if sum(row.notnull()) == 0:
        station_x.append(np.zeros(len(total_stations))) # 공정을 거치지 않은 데이터도 포함시켜야 한다.
    
    else:
        not_null_columns = X.columns[row.notnull()]
        stations = pd.Series(not_null_columns).str.split('_', expand=True).iloc[:, 1].drop_duplicates().to_list() # 결측치가 아닌 컬럼 리스트
        station_x.append(np.isin(total_stations, stations)) # 모든 station별로 True/False

station_x = pd.DataFrame(station_x, index=X.index, columns=total_stations)
station_x = station_x.astype(int) # True = 1, False = 0 변환

station_x

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S40,S41,S43,S44,S45,S47,S48,S49,S50,S51
Id,,,,,,,,,,,,,,,,,,,,,
4,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
11,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19912,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19915,1,1,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
19917,1,1,1,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### 측정된 값의 통계량 추출

이상치 전처리 함수

In [32]:
def remove_outliers(val, w=1.5):
    Q1 = np.quantile(val, 0.25)
    Q3 = np.quantile(val, 0.75)
    IQR = Q3 - Q1

    low_cond = Q1 - w * IQR < val
    high_cond = Q3 + w * IQR > val
    total_cond = np.logical_and(low_cond, high_cond)

    return val[total_cond]

- Id별 통계량 추출 함수 : 평균, 분산, 최대값, 최소값, 첨도, 

In [33]:
def extract_statistical_feature(val): # val : Id별
    if val.notnull().sum() == 0:
        return pd.Series([0] * 6)
    else:
        val = val.copy().dropna() # 결측치 저리
        val = remove_outliers(val) # 이상치 처리

        val_mean = val.mean() # 평균
        val_var = val.var() # 분산
        val_max = val.max() # 최대값
        val_min = val.min() # 최소값
        val_kurtosis = stats.kurtosis(val) # 첨도
        val_rms = np.sqrt(sum(val**2 / len(val))) # 제곱평균제곱근 : Id별 특징이나 경향을 나타내는 대표값 중 하나
    
    return pd.Series([val_mean, val_var, val_max, val_min, val_kurtosis, val_rms])

In [34]:
state_feature_x = X.apply(extract_statistical_feature, axis=1) # axis = 0:index 방향, 1:column 방향
state_feature_x.rename({0:'mean', 1:'variance', 2:'max', 3:'min', 4:'kurtosis', 5:'RMS'}, axis=1, inplace=True) # axis = 0:index, 1:column

In [35]:
state_feature_x

,mean,variance,max,min,kurtosis,RMS
Id,,,,,,
4,-0.013346,0.003363,0.118,-0.164,0.415134,0.059297
6,-0.002573,0.001208,0.096,-0.088,0.632356,0.034741
7,0.001774,0.000579,0.064,-0.060,1.093244,0.024048
9,-0.000248,0.000928,0.079,-0.084,0.837443,0.030357
11,-0.014000,0.003128,0.118,-0.157,0.315412,0.057489
...,...,...,...,...,...,...
19912,0.016471,0.002860,0.144,-0.115,0.126258,0.055807
19915,0.002671,0.002858,0.134,-0.140,0.808986,0.053348
19917,-0.002669,0.001658,0.108,-0.105,1.050556,0.040675


- 공정 데이터와 통계량 데이터 통합

In [36]:
numeric_x = pd.merge(station_x, state_feature_x, left_index=True, right_index=True)
del X

In [37]:
numeric_x.head()

,S0,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S48,S49,S50,S51,mean,variance,max,min,kurtosis,RMS
Id,,,,,,,,,,,,,,,,,,,,,
4,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,-0.013346,0.003363,0.118,-0.164,0.415134,0.059297
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.002573,0.001208,0.096,-0.088,0.632356,0.034741
7,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0.001774,0.000579,0.064,-0.060,1.093244,0.024048
9,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,-0.000248,0.000928,0.079,-0.084,0.837443,0.030357
11,1,1,0,1,1,0,0,1,1,0,...,0,0,0,0,-0.014000,0.003128,0.118,-0.157,0.315412,0.057489


## 범주형 데이터

In [7]:
df = categorical.copy()
df.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.set_index('Id', inplace=True)

### Id별 결측치가 아닌 값 확인

In [10]:
df.iloc[0].dropna().unique()

array([], dtype=object)

In [12]:
df.iloc[2].dropna().unique()

array(['T1'], dtype=object)

In [14]:
df.iloc[50].dropna().unique()

array(['T1', 'T145'], dtype=object)

In [16]:
codes = []

for idx, row in df.iterrows():
    for code in row.dropna().unique():
        if code not in codes:
            codes.append(code)

print(codes)

['T1', 'T3', 'T145', 'T4', 'T143', 'T16', 'T2', 'T256', 'T65536', 'T5', 'T128', 'T16777557', 'T512', 'T16777232', 'T786432', 'T8', 'T-2147482816', 'T1372', 'T1310', 'T6', 'T7', 'T-2147483648', 'T48576', 'T48', 'T9', 'T-21474819', 'T97', 'T32', 'T32896']


In [17]:
codes

['T1',
 'T3',
 'T145',
 'T4',
 'T143',
 'T16',
 'T2',
 'T256',
 'T65536',
 'T5',
 'T128',
 'T16777557',
 'T512',
 'T16777232',
 'T786432',
 'T8',
 'T-2147482816',
 'T1372',
 'T1310',
 'T6',
 'T7',
 'T-2147483648',
 'T48576',
 'T48',
 'T9',
 'T-21474819',
 'T97',
 'T32',
 'T32896']

### T 코드가 등장한 스테이션 확인